# Quantization on MNIST data set, 2d attempt 

#### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

#### Data

In [12]:
# Using tf MNIST data set this time 
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

## 1. Post-training integer quantization

In [13]:
path = './model1_mnist/'
converter = tf.lite.TFLiteConverter.from_saved_model(path)

In [44]:
# Conversion set-up 

converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]

converter.optimizations = [tf.lite.Optimize.DEFAULT]

model = converter.convert()

### No quantization

In [45]:
# Converted model interpreter 

interpreter = tf.lite.Interpreter(model_content=model)
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

test_images = test_images.reshape(10000,784)
interpreter.resize_tensor_input(input_index = input_details[0]['index'],tensor_size=(10000,784))
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], test_images)

interpreter.invoke()
tflite_predictions = interpreter.get_tensor(output_details[0]['index'])

In [46]:
pred = np.argmax(tflite_predictions, axis=1)
print('Accuracy : ', (pred == test_labels).mean() )

Accuracy :  0.9825


In [79]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(data).batch(1).take(1000):
    yield [input_value]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

quant_model = converter.convert()

In [84]:
data = pd.DataFrame(train_images.reshape(60000, 784))
#data
input_details['dtype']

numpy.uint8

In [85]:
input_details = interpreter.get_input_details()[0]
if input_details['dtype'] == np.uint8:
    input_scale, input_zero_point = input_details["quantization"]
    
    testarray2 = test_images_int8 / input_scale
    testarray2 = test_images_int8 + input_zero_point

In [89]:
# Interpreter

interpreter = tf.lite.Interpreter(model_content=quant_model)
interpreter.resize_tensor_input(input_index = input_details['index'],tensor_size=(10000,784))
interpreter.allocate_tensors()

test_images_int8 = np.array(test_images, dtype=np.uint8)
interpreter.set_tensor(input_details['index'], testarray2)
interpreter.invoke()
tflite_predictions2 = interpreter.get_tensor(output_details[0]['index'])

pred2 = np.argmax(tflite_predictions2, axis=1)
print('Accuracy : ', (pred2 == test_labels).mean() )

Accuracy :  0.089


In [ ]:
input_shape = input_details[0]['shape']
print(input_shape)

# Is set to 0 at the beginning
print(interpreter.get_tensor(output_details[0]['index']))

test_img = df_test.loc[35008].values # is a 9 
input_data = np.array(test_img, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data.reshape(1,784))

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data) # predicts a 9 too 


In [ ]:
converter2.optimizations = [tf.lite.Optimize.DEFAULT]
converter2.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter2.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 
converter2.inference_input_type = tf.uint8
converter2.inference_output_type = tf.uint8